In [2]:
import numpy as np
import pandas as pd

%matplotlib inline

from datetime import date,datetime,timedelta
from dateutil.parser import parse

import json
import re
from glob import glob
from IPython.display import Image

import seaborn as sns
import folium as g

import matplotlib.pyplot as plt
from matplotlib import rc
# 한글 폰트 문제 해결
rc('font', family='AppleGothic')
# 차트 축 <- 음수 부호 지원
plt.rcParams['axes.unicode_minus'] = False

In [101]:
import sklearn
from sklearn.datasets import load_iris, load_breast_cancer

from sklearn.model_selection import train_test_split,KFold,StratifiedKFold , cross_val_score, cross_validate,GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble  import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, make_scorer, precision_recall_curve

from sklearn.base import BaseEstimator

from sklearn.preprocessing import LabelEncoder,OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import missingno as msno

#### 정확도 : 실 데이터와 예측 데이터가 얼마나 같은지를 판단하는 자료

- 문제점 : 이진분류의 경우 모델의 성능을 왜곡할수 있다
- why?:
- solution : f1-score

#### 분류모델 성능평가를 위한 이해 

In [8]:
target = [1,0,1,1,1,0,1,0,0,0,1]
predict = [1,1,1,1,0,0,0,0,1,1,1]

tp = tn = fn = fp = 0
for i in range(len(target)):
    # TP
    if target[i]==1 and predict[i]==1:
        tp+=1
    # TN
    elif target[i]==0 and predict[i]==0:
        tn+=1
    # FN
    elif target[i]==1 and predict[i]==0:
        fn+=1
    # FP
    elif target[i]==0 and predict[i]==1:
        fp+=1
        
print('TP:',tp)
print('TN:',tn)
print('FN:',fn)
print('FP:',fp)

TP: 4
TN: 2
FN: 2
FP: 3


In [10]:
confusion_matrix(target,predict)

array([[2, 3],
       [2, 4]])

In [12]:
accuracy_score(target,predict)

0.5454545454545454

In [13]:
precision_score(target,predict)

0.5714285714285714

In [14]:
recall_score(target,predict)

0.6666666666666666

In [11]:
f1_score(target,predict)

0.6153846153846153

In [ ]:
# 이진분류
# 제품을 생상하는 제조공장에서 품질 테스트를 실시하여 불량품을 찾아내고 불량품을 공장으로 돌려보낸다

# 품질테스스 결과가 양성 ->불량품을 예측 음성->정상제품으로 예측

# TP : 불량품이라고 예측한것중 실제 불량품인것
# TN : 정상이라고 에측한것중 실제 정상인것
# 

1. 정밀도(precision) : TP/(FP+TP)
- positive로 예측한 값들 중 실제 값이 postitive인 비율
- 스팸메일
2.  재현율(Recall) : TP/(FN+TP)
- 실제 값이 postivie인 값들 중에 예측과 실제값이 postive로 일치하는 비율
- 의학(암진단) 금융(사기 판별)

* 정밀도와 재현율 사이에도 trade-off 존재

In [19]:
titanic_df = pd.read_csv('./Data/titanic_train.csv')
titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


- 정확도의 문제 : 이진 분류의 경우 데이터의 구성에 따라 모델의 성능이 왜곡될수 있다

In [30]:
class UserEstimator(BaseEstimator):
    
    def fit(self,X,y=None):
        pass
    
    def predict(self,X):
        pred = np.zeros(X.shape[0],1)
        for idx in range(X.shape[0]):
            if X['Sex'].iloc[idx]==1:
                pred[idx] = 0
            else: 
                pred[idx] = 1
        return pred
        

In [21]:
# 레이블 인코딩
# 레일블 인코딩 Cabin , Sex , Embarked
def drop_features(frm):
    frm.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return frm


def pre_processing(frm):
    frm['Age'].fillna(frm['Age'].mean(), inplace=True)
    frm['Cabin'].fillna('N', inplace=True)
    frm['Embarked'].fillna('N', inplace=True)
    return frm


def label_encoder(frm):
    frm['Cabin'] = frm['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        encoder = LabelEncoder()
        encoder.fit(frm[feature])
        frm[feature] = encoder.transform(frm[feature])
    return  frm

def transform_features(frm):
    frm = drop_features(frm)
    frm = pre_processing(frm)
    frm = label_encoder(frm)
    return frm

In [22]:
titanic_label = titanic_df['Survived']
titanic_features = titanic_df.drop('Survived',axis=1)

print(titanic_label)
print(titanic_features)

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64
     PassengerId  Pclass                                               Name  \
0              1       3                            Braund, Mr. Owen Harris   
1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2              3       3                             Heikkinen, Miss. Laina   
3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4              5       3                           Allen, Mr. William Henry   
..           ...     ...                                                ...   
886          887       2                              Montvila, Rev. Juozas   
887          888       1                       Graham, Miss. Margaret Edith   
888          889       3           Johnston, Miss. Catherine Helen "Carrie"   
889          890       1                              Behr, Mr. Karl 

In [23]:
subset_features = transform_features(titanic_features)
subset_features

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,22.000000,1,0,7.2500,7,3
1,1,0,38.000000,1,0,71.2833,2,0
2,3,0,26.000000,0,0,7.9250,7,3
3,1,0,35.000000,1,0,53.1000,2,3
4,3,1,35.000000,0,0,8.0500,7,3
...,...,...,...,...,...,...,...,...
886,2,1,27.000000,0,0,13.0000,7,3
887,1,0,19.000000,0,0,30.0000,1,3
888,3,0,29.699118,1,2,23.4500,7,3
889,1,1,26.000000,0,0,30.0000,2,0


In [24]:
X_train, X_test, y_train, y_test = train_test_split(subset_features,titanic_label,test_size=0.2,random_state=100)

In [27]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 8), (179, 8), (712,), (179,))

In [28]:
type(X_train), type(X_test), type(y_train), type(y_test)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series)

In [32]:
user_model = UserEstimator()
user_model.fit(X_train,y_train)
y_pred = user_model.predict(X_test)

print('accuracy',accuracy_score(y_test,y_pred))

TypeError: Cannot interpret '1' as a data type

In [43]:
def metrics_eval(target,predict):
    print(confusion_matrix(target,predict))
    print('accuracy:',accuracy_score(target,predict))
    print('precision:',precision_score(target,predict))
    print('recall:',recall_score(target,predict))
    print('f1-score:',f1_score(target,predict))

In [44]:
lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
y_predict = lr_model.predict(X_test)
metrics_eval(y_test,y_predict)

[[90 14]
 [21 54]]
accuracy: 0.8044692737430168
precision: 0.7941176470588235
recall: 0.72
f1-score: 0.7552447552447551


/Users/iganghui/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [46]:
print('accuracy:',(90+54)/(90+14+21+54))
print('precision:',54/(14+54))
print('recall:',54/(21+54))

accuracy: 0.8044692737430168
precision: 0.7941176470588235
recall: 0.72


In [50]:
# 교차검증
fold = KFold(n_splits=20)
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall':make_scorer(recall_score),
           'f1-score':make_scorer(f1_score)}


result = cross_validate(lr_model,X_train,y_train,cv=fold,scoring=scoring)

/Users/iganghui/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/iganghui/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

In [52]:
print('accuracy:',result['test_accuracy'].mean())
print('precision:',result['test_precision'].mean())
print('recall:',result['test_recall'].mean())
print('f1-score:',result['test_f1-score'].mean())

accuracy: 0.7895634920634922
precision: 0.7316997456877488
recall: 0.716394863306628
f1-score: 0.7063641038192401


#### 유방암 데이터 : 정확<재현율
- 모든 양성인 환자들을 기준으로 양성이라고 예측한 비율이 높을수록 좋은모델이라고 판단할수 있다

In [55]:
cancer = load_breast_cancer()
print(type(cancer))
cancer

<class 'sklearn.utils.Bunch'>


{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [58]:
cancer_df = pd.DataFrame(data=cancer.data,columns=cancer.feature_names)
cancer_df['target'] = cancer.target

cancer_df.head()
cancer_df['target'].value_counts()


1    357
0    212
Name: target, dtype: int64

In [59]:
cancer_label = cancer_df['target']
cancer_features = cancer_df.drop('target',axis=1)

In [61]:
X_train, X_test,y_train,y_test = train_test_split(cancer_features,cancer_label,test_size=0.2,random_state=100)
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
metrics_eval(y_test,y_pred)

[[47  2]
 [ 1 64]]
accuracy: 0.9736842105263158
precision: 0.9696969696969697
recall: 0.9846153846153847
f1-score: 0.9770992366412214


In [92]:
X_train, X_test,y_train,y_test = train_test_split(cancer_features,cancer_label,test_size=0.2,random_state=100)
lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
y_pred = lr_model.predict(X_test)
metrics_eval(y_test,y_pred)

[[46  3]
 [ 3 62]]
accuracy: 0.9473684210526315
precision: 0.9538461538461539
recall: 0.9538461538461539
f1-score: 0.9538461538461539


/Users/iganghui/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [93]:
# 교차검증 cross_valide 후 성능평가
fold = KFold(n_splits=20)
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score),
           'recall':make_scorer(recall_score),
           'f1-score':make_scorer(f1_score)}

result = cross_validate(rf_model,X_train,y_train,cv=fold,scoring=scoring)
print('accuracy:',result['test_accuracy'].mean())
print('precision:',result['test_precision'].mean())
print('recall:',result['test_recall'].mean())
print('f1-score:',result['test_f1-score'].mean())

accuracy: 0.958399209486166
precision: 0.953672325481536
recall: 0.9819444444444445
f1-score: 0.9660963995200031


In [ ]:
# 랜덤포레스트
# n_estimators : 생성할 tree의 갯수를 의미
# max_featuers : 선택할 feature의 수 
# max_depth : 최대 선택할 트리의 깊이

In [97]:
# GridSerachCV
X_train, X_test,y_train,y_test = train_test_split(cancer_features,cancer_label,test_size=0.2,random_state=100)
rf_model = RandomForestClassifier()

param = {
    'n_estimators':[200,300],
    'max_features':[10,11],
    'max_depth':[5,6]
}

grid_rf_model = GridSearchCV(rf_model,
                              param_grid=param,
                              cv=20,
                              scoring = 'recall',
                              refit=True)

grid_rf_model.fit(X_train,y_train)
grid_rf_model.cv_results_

{'mean_fit_time': array([0.26502343, 0.39485612, 0.2765746 , 0.41359693, 0.2697603 ,
        0.40398116, 0.282415  , 0.4247148 ]),
 'std_fit_time': array([0.00903684, 0.00393099, 0.00321161, 0.00457788, 0.00335989,
        0.00383892, 0.00309525, 0.00374639]),
 'mean_score_time': array([0.0095253 , 0.01353163, 0.00958247, 0.01353704, 0.00956314,
        0.01377612, 0.00961288, 0.01358869]),
 'std_score_time': array([0.00027284, 0.00024595, 0.0002938 , 0.0002786 , 0.00017959,
        0.0002607 , 0.00023738, 0.00033364]),
 'param_max_depth': masked_array(data=[5, 5, 5, 5, 6, 6, 6, 6],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_features': masked_array(data=[10, 10, 11, 11, 10, 10, 11, 11],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[200, 300, 200, 300, 200, 300, 200, 

In [99]:
print(grid_rf_model.best_params_)
print(grid_rf_model.best_score_)

{'max_depth': 5, 'max_features': 10, 'n_estimators': 200}
0.9864285714285714


In [100]:
y_predict = grid_rf_model.best_estimator_.predict(X_test)
metrics_eval(y_test,y_predict)

[[47  2]
 [ 1 64]]
accuracy: 0.9736842105263158
precision: 0.9696969696969697
recall: 0.9846153846153847
f1-score: 0.9770992366412214


#### precision_recall_curve (실제 값, 예측 확률 값) : 임계값 변화에 따른 평가 지표 값을 반환
- 반환